In [1]:
### data wrangling - combining files into one single file (one line of information per each user)

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set()

In [3]:
# Reading the Credit Bureau file from csv
bureau = pd.read_csv('bureau.csv', index_col=None, engine='python')

KeyboardInterrupt: 

In [ ]:
bureau.head()

In [ ]:
# Reading the Bureau Balance file from csv
bureau_balance = pd.read_csv('bureau_balance.csv', index_col=None, engine='python')

In [ ]:
bureau_balance.head()

In [ ]:
bureau_balance = bureau_balance.join(pd.get_dummies(bureau_balance.STATUS)).drop("STATUS", axis=1)

In [ ]:
bureau_balance = bureau_balance.drop("MONTHS_BALANCE", axis=1)

In [ ]:
bureau_balance.head()

In [ ]:
counts = bureau_balance['SK_ID_BUREAU'].value_counts().to_dict()


In [ ]:
bureau_balance_grouped = bureau_balance.groupby(['SK_ID_BUREAU'], as_index=False).mean()

In [ ]:
bureau_balance_grouped.head()

In [ ]:
bureau_balance_grouped['Month_Balance_Count'] = bureau_balance_grouped['SK_ID_BUREAU'].map(counts)
bureau_balance_grouped.head()

In [ ]:
# Back to the Credit Bureau file - transforming categorical variables into Dummies

bureau = bureau.join(pd.get_dummies(bureau.CREDIT_ACTIVE)).drop("CREDIT_ACTIVE", axis=1)
bureau = bureau.join(pd.get_dummies(bureau.CREDIT_CURRENCY)).drop("CREDIT_CURRENCY", axis=1)
bureau = bureau.join(pd.get_dummies(bureau.CREDIT_TYPE)).drop("CREDIT_TYPE", axis=1)

bureau.head()

In [ ]:
bureau.shape

In [ ]:
# Merging the Credit Bureau file with the Balance file
bureau = bureau.merge(bureau_balance_grouped, on=('SK_ID_BUREAU'), how='left')

In [ ]:
bureau.shape

In [ ]:
bureau_grouped = bureau.groupby(['SK_ID_CURR'], as_index=False).sum()

In [ ]:
bureau_grouped.head()

In [ ]:
bureau_grouped.shape

In [ ]:
#Validating unique ID count against the size of final grouped DataFrame
bureau_grouped['SK_ID_CURR'].nunique()

In [ ]:
bureau_grouped = bureau_grouped.drop(['SK_ID_BUREAU'], axis=1)

In [ ]:
# Reading the Credit Card Balance file from csv
cc_balance = pd.read_csv('credit_card_balance.csv', index_col=None, engine='python')

In [ ]:
cc_balance.head()

In [ ]:
# Transforming categorical variable into Dummies

cc_balance = cc_balance.join(pd.get_dummies(cc_balance.NAME_CONTRACT_STATUS)).drop("NAME_CONTRACT_STATUS", axis=1)

cc_balance.head()

In [ ]:
cc_balance.shape

In [ ]:
cc_balance['SK_ID_PREV'].nunique()

In [ ]:
# Creating unique combinations of the previous loans IDs with the current loan IDs
cc_balance_prev_curr = cc_balance[['SK_ID_PREV','SK_ID_CURR']]
cc_balance_prev_curr.info()

In [ ]:
cc_balance_prev_curr = cc_balance_prev_curr.groupby(['SK_ID_PREV'], as_index=False).mean()
cc_balance_prev_curr.tail()

In [ ]:
# Dropping Current ID variable - later will re-merge
cc_balance = cc_balance.drop("SK_ID_CURR", axis=1)

In [ ]:
# Summing past Installement payment information and grouping by previous ID.
cc_balance = cc_balance.groupby(['SK_ID_PREV'], as_index=False).sum()
cc_balance.head()

In [ ]:
# Merging back the unique SK_ID_CURR variable to the installments_payments DataFrame
cc_balance = cc_balance.merge(cc_balance_prev_curr, on=('SK_ID_PREV'), how='left')
cc_balance.head()

In [ ]:
cc_balance['SK_ID_PREV'].nunique()

In [ ]:
# Reading the Previous Applications file from csv
prev_app = pd.read_csv('previous_application.csv', index_col=None, engine='python')

In [ ]:
prev_app.head()

In [ ]:
# Previous Applications - transforming categorical variables into Dummies

prev_app = prev_app.join(pd.get_dummies(prev_app.NAME_CONTRACT_TYPE)).drop("NAME_CONTRACT_TYPE", axis=1)
prev_app = prev_app.join(pd.get_dummies(prev_app.WEEKDAY_APPR_PROCESS_START)).drop("WEEKDAY_APPR_PROCESS_START", axis=1)
prev_app = prev_app.join(pd.get_dummies(prev_app.NAME_SELLER_INDUSTRY), rsuffix='Seller_Ind').drop("NAME_SELLER_INDUSTRY", axis=1)
prev_app = prev_app.join(pd.get_dummies(prev_app.NAME_YIELD_GROUP), rsuffix='_Name_Group').drop("NAME_YIELD_GROUP", axis=1)
prev_app = prev_app.join(pd.get_dummies(prev_app.PRODUCT_COMBINATION)).drop("PRODUCT_COMBINATION", axis=1)
prev_app = prev_app.join(pd.get_dummies(prev_app.FLAG_LAST_APPL_PER_CONTRACT)).drop("FLAG_LAST_APPL_PER_CONTRACT", axis=1)

prev_app.head()

In [ ]:
prev_app.shape

In [ ]:
prev_app['SK_ID_PREV'].nunique()

prev_app has the same number of unique SK_ID_PREV identifiers as there are rows in the DataFrame

In [ ]:
# Reading the Previous Applications file from csv
pos_cash_balance = pd.read_csv('POS_CASH_BALANCE.csv', index_col=None, engine='python')

In [ ]:
pos_cash_balance.head()

In [ ]:
# Previous Applications - transforming categorical variables into Dummies

pos_cash_balance = prev_app.join(pd.get_dummies(pos_cash_balance.NAME_CONTRACT_STATUS), \
                                 rsuffix='_Name_Status').drop("NAME_CONTRACT_STATUS", axis=1)

pos_cash_balance.head()

In [ ]:
pos_cash_balance.shape

In [ ]:
pos_cash_balance['SK_ID_PREV'].nunique()

pos cash balance has the same number of unique SK_ID_PREV identifiers as there are rows in the DataFrame

In [ ]:
# Reading the Installments Payments file from csv
installments_payments = pd.read_csv('installments_payments.csv', index_col=None, engine='python')

In [ ]:
installments_payments.head()

In [ ]:
print(installments_payments.shape)

In [ ]:
# Creating unique combinations of the previous loans IDs with the current loan IDs
installments_payments_prev_curr = installments_payments[['SK_ID_PREV','SK_ID_CURR']]
installments_payments_prev_curr = installments_payments_prev_curr.groupby(['SK_ID_PREV'], as_index=False).mean()
installments_payments_prev_curr.tail()

In [ ]:
# Dropping Current ID variable - later will re-merge
installments_payments = installments_payments.drop("SK_ID_CURR", axis=1)


In [ ]:
# New Variable for a flag of a late payment relative to when it was supposed to have been paid
installments_payments['late'] = installments_payments['DAYS_ENTRY_PAYMENT'] > installments_payments['DAYS_INSTALMENT']
installments_payments['late'] = installments_payments['late'].astype(int)

In [ ]:
# Summing past Installement payment information and grouping by previous ID.
installments_payments = installments_payments.groupby(['SK_ID_PREV'], as_index=False).sum()
installments_payments.head()

In [ ]:
installments_payments.shape

In [ ]:
installments_payments['SK_ID_PREV'].nunique()

In [ ]:
# Merging back the unique SK_ID_CURR variable to the installments_payments DataFrame
installments_payments = installments_payments.merge(installments_payments_prev_curr, on=('SK_ID_PREV'), how='left')
installments_payments.head()

Installments payments now has a unique SK_ID_PREV for every row

In [ ]:
#Dropping Curr ID from merged files to keep a single column
pos_cash_balance = pos_cash_balance.drop(['SK_ID_CURR'], axis=1)
installments_payments = installments_payments.drop(['SK_ID_CURR'], axis=1)
cc_balance = cc_balance.drop(['SK_ID_CURR'], axis=1)

In [ ]:
# Merging prev_app with installments_payments
prev_app_and_installments = prev_app.merge(installments_payments, on=('SK_ID_PREV'), how='left')
prev_app_and_installments.head()

In [ ]:
# Merging prev_app+installments_payments with pos_cash_balance
prev_app_and_installments_pos_cash = prev_app_and_installments.merge(pos_cash_balance, on=('SK_ID_PREV'), how='left')
prev_app_and_installments_pos_cash.head()

In [ ]:
# Merging prev_app+installments_payments+pos_cash_balance with cc_balance
all_prev_data = prev_app_and_installments_pos_cash.merge(cc_balance, on=('SK_ID_PREV'), how='left')
all_prev_data.head()

In [ ]:
# Checking that removal of Curr_ID Columns worked and we have a single Curr_ID Column for subsequent merging
all_prev_data.columns = all_prev_data.columns.map(str)
filter_col = all_prev_data.loc[:, all_prev_data.columns.str.startswith('SK_ID_CURR')]
filter_col.head()

In [ ]:
filter_col['SK_ID_CURR'].nunique()

In [ ]:
all_prev_data['SK_ID_CURR'].nunique()

In [ ]:
all_prev_data.shape

In [ ]:
# Summing past Installement payment information and grouping by Current ID (Combining info on past transactions)
all_prev_data_by_ID_CURR = all_prev_data.groupby(['SK_ID_CURR'], as_index=False).sum()
all_prev_data_by_ID_CURR = all_prev_data_by_ID_CURR.drop(['SK_ID_PREV'], axis=1)
all_prev_data_by_ID_CURR.head()

In [ ]:
all_prev_data_by_ID_CURR.shape

In [ ]:
# Read Train Data Set
application_train = pd.read_csv('application_train.csv', index_col=None, engine='python')

In [ ]:
# Merging grouped previous transactions and bureau files with the train data set
application_train_merged = application_train.merge(all_prev_data_by_ID_CURR, on=('SK_ID_CURR'), how='left')
application_train_merged = application_train_merged.merge(bureau_grouped, on=('SK_ID_CURR'), how='left')
application_train_merged.head()

### Now looking at missing data to decide which columns to drop and which to fill in data for

In [ ]:
# renaming a duplicate named variable
cols = []
count = 1
for column in application_train_merged.columns:
    if column == 'AMT_ANNUITY_x':
        cols.append('AMT_ANNUITY_x_'+str(count))
        count+=1
        continue
    cols.append(column)
application_train_merged.columns = cols

In [ ]:
# renaming a duplicate named variable
cols = []
count = 1
for column in application_train_merged.columns:
    if column == 'AMT_ANNUITY_y':
        cols.append('AMT_ANNUITY_y_'+str(count))
        count+=1
        continue
    cols.append(column)
application_train_merged.columns = cols

In [ ]:
# Finding the number of missing variables by column 

def Missing_variables(df):
    nan_values = []
    for i in df:
        count_nan = len(df[i]) - df[i].count()
        nan_values.append(count_nan)
    percentage = [x / len(df)*100 for x in nan_values]
    nan_df = list(df.columns.values)
    percentage_list = pd.DataFrame(
    {'columns': nan_df,
     'number_Nan': nan_values,
     'percentage': percentage
    })
    
    Nan_Ascending = percentage_list.loc[percentage_list['percentage'] != 0].sort_values('percentage', ascending=False)
    
    return Nan_Ascending

In [ ]:
# Before removing variables with high Nan values, making exception to EXT_SOURCE_1
# This is after seeing its high feature importance to the model
application_train_merged['EXT_SOURCE_1'].fillna((application_train_merged['EXT_SOURCE_1'].mean()), inplace=True)

In [ ]:
Missing_var = Missing_variables(application_train_merged)
print('There are', len(Missing_var), 'columns with missing variables out of', \
      len(application_train_merged.columns), 'columns in the dataframe')
Missing_var.head(10)

In [ ]:
# Creating list of column names for columns with over 50% of missing variables
# Removing these columns from the application_train_merged data set

Mising_var_high = Missing_var[Missing_var['percentage']>35]
high_Nan_column_names = Mising_var_high['columns'].tolist()

application_train_merged = application_train_merged.drop(high_Nan_column_names, axis=1)

In [ ]:
Missing_var = Missing_variables(application_train_merged)
print('There are', len(Missing_var), 'columns with missing variables out of', \
      len(application_train_merged.columns), 'columns in the dataframe')
Missing_var.head(20)

In [ ]:
# The majority of missing values are 5.350703 percentage, and those appear on those that did not merge 
# with the applicaiton_train data.
# Since there is much incomplete data - those 5% of the data will be removed for the model data-sets
# Using "late" variable for the removal. 

application_train_merged = application_train_merged[np.isfinite(application_train_merged['late'])]

In [ ]:
Missing_var = Missing_variables(application_train_merged)
print('There are', len(Missing_var), 'columns with missing variables out of', \
      len(application_train_merged.columns), 'columns in the dataframe')
Missing_var.head(10)

In [ ]:
# The next majority of missing values are 14.275554 percentage, and those appear on those that did not merge 
# with the applicaiton_train data from the Bureau data.
# Since there is much incomplete data - those 14% of the data will be removed for the model data-sets
# Using "Closed" variable for the removal. 

application_train_merged = application_train_merged[np.isfinite(application_train_merged['Closed'])]

In [ ]:
Missing_var = Missing_variables(application_train_merged)
print('There are', len(Missing_var), 'columns with missing variables out of', \
      len(application_train_merged.columns), 'columns in the dataframe')
Missing_var.head(10)

In [ ]:
# Filling NaN values in OCCUPATION_TYPE as "Unemployed"
application_train_merged['OCCUPATION_TYPE'].fillna('Unemployed', inplace=True)

# Filling NaN values in EXT_Source_3 and _2 with the mean of each.
application_train_merged['EXT_SOURCE_3'].fillna((application_train_merged['EXT_SOURCE_3'].mean()), inplace=True)
application_train_merged['EXT_SOURCE_2'].fillna((application_train_merged['EXT_SOURCE_2'].mean()), inplace=True)

# Filling NaN values in NAME_TYPE_SUITE as "Unaccompanied"
application_train_merged['NAME_TYPE_SUITE'].fillna('Unaccompanied', inplace=True)

# Filling NaN values in XXX_CNT_SOCIAL_CIRCLE as "0.0". Making assumption that not listed is not observed.
# These are defined as "How many observation of client's social surroundings defaulted on 30 DPD (days past due)""
application_train_merged['OBS_30_CNT_SOCIAL_CIRCLE'].fillna('0.0', inplace=True)
application_train_merged['DEF_30_CNT_SOCIAL_CIRCLE'].fillna('0.0', inplace=True)
application_train_merged['OBS_60_CNT_SOCIAL_CIRCLE'].fillna('0.0', inplace=True)
application_train_merged['DEF_60_CNT_SOCIAL_CIRCLE'].fillna('0.0', inplace=True)

# Filling NaN values in EXT_Source_3 and _2 with the mean of each.
application_train_merged['AMT_GOODS_PRICE'].fillna((application_train_merged['AMT_GOODS_PRICE'].mean()), inplace=True)
application_train_merged['AMT_ANNUITY_x_1'].fillna((application_train_merged['AMT_ANNUITY_x_1'].mean()), inplace=True)

In [ ]:
Missing_var = Missing_variables(application_train_merged)
print('There are', len(Missing_var), 'columns with missing variables out of', \
      len(application_train_merged.columns), 'columns in the dataframe')
Missing_var.head(10)

### Moving to look at outlier values... Let's save & move to EDA

In [ ]:
application_train_merged.to_csv('application_train_merged.csv',index=False)

In [ ]:
application_train_merged.describe()

In [ ]:
application_train_merged.TARGET.sum()